In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from peft import PeftModel
import torch
from trl import setup_chat_format

In [2]:
model_name = "NousResearch/Meta-Llama-3.1-8B-Instruct"
new_model = "Llama3.1-ruFinGPT"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

C:\Users\Sergey\.conda\envs\edok\Lib\site-packages\peft\tuners\lora\bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [7]:
messages = [
    {
        "role": "system", 
        "content": "Какая тональность у этой новости? Пожалуйста, выберите вариант ответа из {сильно негативно/умеренно негативно/слабо негативно/нейтрально/слабо позитивно/умеренно позитивно/сильно позитивно}."
    },   
    {
        "role": "user",
        "content": "Чистая прибыль РусГидро по РСБУ за 1 полугодие выросла на 17%"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)
pipe = pipeline(
    "text-generation",
    model=base_model_reload,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True, temperature=1, top_k=1, top_p=1)
print(outputs[0]["generated_text"])

<|im_start|>assistant
Какая тональность у этой новости? Пожалуйста, выберите вариант ответа из {сильно негативно/умеренно негативно/слабо негативно/нейтрально/слабо позитивно/умеренно позитивно/сильно позитивно}.<|im_end|>
<|im_start|>user
Чистая прибыль РусГидро по РСБУ за 1 полугодие выросла на 17%<|im_end|>
<|im_start|>assistant

<|im_start|>assistant
Какая тональность у этой новости? Пожалуйста, выберите вариант ответа из {сильно негативно/умеренно негативно/слабо негативно/нейтрально/слабо позитивно/умеренно позитивно/сильно позитивно}.<|im_end|>
<|im_start|>user
Чистая прибыль РусГидро по РСБУ за 1 полугодие выросла на 17%<|im_end|>
<|im_start|>assistant
умеренно позитивно
assistant
Учитывая, что РусГидро является крупнейшим производителем электроэнергии в России, рост чистой прибыли на 17% в первом полугодии 2022 года является значительным достижением. Это связано с тем, что компания продолжает увеличивать производство электроэнергии, а также снизила затраты на топливо и другие 